# CityBikes

Send a request to CityBikes for the city of your choice. 

In [2]:
# Importing necessary libraries
import pandas as pd
import requests
import json
import os

In [3]:
# Accessing environmental variables
yelp_key = os.environ.get('yelp_api')
yelp_key2 = os.environ.get('yelp_api2')
FOURSQUARE_KEY = os.environ.get('FOURSQUARE_API_KEY')

Through the information from *The Philadelphia Inquirer* (https://www.inquirer.com/news/worst-traffic-roads-philadelphia.html#:~:text=How%20to%20beat%20Philly%20traffic,avoid%20traffic%2C%20according%20to%20INRIX.), I noted that peak travel times are around 3 - 6pm in the afternoon (1 - 4pm here in Calgary).

So I used a random time generator (https://www.random.org/clock-times/), inputed the start and end times, used the output - **2:30pm** - I placed my API call at that time, just for a slightly better picture of the usage.

Using empty slots as an indicator for bike usage.

In [4]:
# Calling the API for Philadelphia City Bikes.
base_url = "http://api.citybik.es/v2/networks"
city_name = "indego"  # The network_id for Philadephia

# Construct the URL for Philly network.
philly_url = f"{base_url}/{city_name}"

# Make the API request.
response = requests.get(philly_url)
response

<Response [200]>

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [5]:
philly = response.json()
# philly

In [7]:
# Saving the json to have a consistent file during analysis.
json_file_path = 'philly.json'

with open(json_file_path, 'w') as json_file:
    json.dump(philly, json_file)

In [6]:
# Read philly JSON data from a file
with open("C:/Users/HP/Music/LHLDataCourse/Python/project_data/philly.json", "r") as json_file:
    philly = json.load(json_file)

Put your parsed results into a DataFrame.

In [7]:
philly['network']['stations'][0].keys()

dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])

In [9]:
# Normalize into a DataFrame.
from pandas.io.json import json_normalize
import pandas as pd

df = pd.json_normalize(philly['network']['stations'])
df.head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.last_updated,extra.rental_uris.android,extra.rental_uris.ios,extra.renting,extra.returning,extra.uid
0,3,15,ebe10cb6ab4ac2abfb050a037fd21d22,39.93431,-75.16042,10th & Federal,2023-09-08T20:24:39.824000Z,1201 S. 10th Street,1694204679,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,0,0,bcycle_indego_3098
1,13,6,5d8556ab318071bbb3d0eb310d1300cd,39.93401,-75.15094,4th & Washington,2023-09-08T20:24:39.825000Z,1110 S. 4th Street,1694204679,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_indego_3099
2,11,3,f639a03c6bc95cbf045611e736857e5f,39.99119,-75.17975,29th & Dauphin,2023-09-08T20:24:39.827000Z,2301 N. 29th Street,1694204679,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_indego_3096
3,6,11,37f346714d732902704277ff7c049d22,39.97888,-75.13339,"Berks Station, MFL",2023-09-08T20:24:39.827000Z,1902 N. Front Street,1694204679,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_indego_3097
4,3,1,7c87d0b867af6cc138c49ad20fd6a9cb,39.98837,-75.18701,33rd & Diamond,2023-09-08T20:24:39.829000Z,2039 N. 33rd Street,1694204679,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_indego_3093


In [10]:
# Select the needed columns into a working file.
selected_columns = ['name', 'empty_slots', 'free_bikes','latitude', 'longitude']

ph_bikes = df.loc[:, selected_columns]
ph_bikes.head()

,name,empty_slots,free_bikes,latitude,longitude
0,10th & Federal,3,15,39.93431,-75.16042
1,4th & Washington,13,6,39.93401,-75.15094
2,29th & Dauphin,11,3,39.99119,-75.17975
3,"Berks Station, MFL",6,11,39.97888,-75.13339
4,33rd & Diamond,3,1,39.98837,-75.18701


In [11]:
# Rename the index column to "ID" to aid with joining later
ph_bikes = ph_bikes.rename_axis("ID").reset_index()
# ph_bikes.head()

In [12]:
csv_file_path = 'ph_bikes.csv'

# Save the DataFrame to a CSV file
ph_bikes.to_csv(csv_file_path, index=False)

In [13]:
# read in saved csv of philly bike stations.
philly_df = pd.read_csv("C:/Users/HP/Music/LHLDataCourse/Python/project_data/ph_bikes.csv")
philly_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           233 non-null    int64  
 1   name         233 non-null    object 
 2   empty_slots  233 non-null    int64  
 3   free_bikes   233 non-null    int64  
 4   latitude     233 non-null    float64
 5   longitude    233 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 11.1+ KB
